In [ ]:
%cd /content/ai-competition-baselines
!python -m src.stage1_binary.infer \
  --config configs/stage1_binary.yaml \
  --data_root "/content/drive/MyDrive/datasets/defects_stage1" \
  --ckpt runs/stage1_binary/best_stage1.pt \
  --test_dir images/test \
  --out submissions/stage1.csv
!head -n 5 submissions/stage1.csv

In [ ]:
# Stage 2 Test Inference
DATA_ROOT = "/content/drive/MyDrive/datasets/defects_stage2"
TEST_DIR = "images/test"
OUT_CSV = "/content/drive/MyDrive/project_stage2/submissions/stage2_test.csv"
CKPT_PATH = "/content/drive/MyDrive/project_stage2/runs/stage2_multilabel/best_stage2.pt"
CONFIG_PATH = "/content/drive/MyDrive/project_stage2/configs/stage2_multilabel.yaml"

!python -m src.stage2_multilabel.infer \
    --config "{CONFIG_PATH}" \
    --data_root "{DATA_ROOT}" \
    --ckpt "{CKPT_PATH}" \
    --test_dir "{TEST_DIR}" \
    --out "{OUT_CSV}"

import pandas as pd
df = pd.read_csv(OUT_CSV)
df.head(10)

In [ ]:
# Stage 2 Validation Inference
VAL_DIR = "images/val"
OUT_CSV = "/content/drive/MyDrive/project_stage2/submissions/stage2_val.csv"

!python -m src.stage2_multilabel.infer \
    --config "{CONFIG_PATH}" \
    --data_root "{DATA_ROOT}" \
    --ckpt "{CKPT_PATH}" \
    --test_dir "{VAL_DIR}" \
    --out "{OUT_CSV}"

df = pd.read_csv(OUT_CSV)
df.head(10)

In [ ]:
# Stage 2 Evaluation on Validation Set
GT_CSV = f"{DATA_ROOT}/val.csv"
PRED_CSV = "/content/drive/MyDrive/project_stage2/submissions/stage2_val.csv"

!python -m src.stage2_multilabel.eval \
    --gt "{GT_CSV}" \
    --pred "{PRED_CSV}"

# With Threshold Tuning
!python -m src.stage2_multilabel.eval \
    --gt "{GT_CSV}" \
    --pred "{PRED_CSV}" \
    --tune_thresholds

In [ ]:
import json
with open('/content/drive/MyDrive/project_stage2/runs/stage2_multilabel/train_summary.json', 'r') as f:
    summary = json.load(f)
print("Training History:")
print("-" * 100)
for epoch_data in summary['history']:
    print(f"Epoch {epoch_data['epoch']}: train_loss={epoch_data['train_loss']:.4f}, val_loss={epoch_data['val_loss']:.4f}, mAP={epoch_data['map']:.4f}, macro_f1={epoch_data['macro_f1']:.4f}")
print("-" * 100)
print(f"Best mAP: {summary['best_primary']:.4f}")

In [ ]:
import matplotlib.pyplot as plt
val_df = pd.read_csv(f'{DATA_ROOT}/val.csv')
pred_df = pd.read_csv('/content/drive/MyDrive/project_stage2/submissions/stage2_val.csv')
label_cols = ["Tucked", "Wrinkles", "Items"]
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for idx, label in enumerate(label_cols):
    gt_counts = val_df[label].value_counts().sort_index()
    axes[idx].bar(['Absent','Present'], gt_counts.values, color=['#3498db','#e74c3c'])
    axes[idx].set_title(f'{label} Distribution', fontweight='bold', fontsize=12)
    axes[idx].set_ylabel('Count')
    axes[idx].grid(axis='y', alpha=0.3)
    for i,v in enumerate(gt_counts.values): axes[idx].text(i,v+5,str(v),ha='center',fontweight='bold')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/project_stage2/label_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Visualize Sample Stage 2 Predictions
from PIL import Image
merged = val_df.merge(pred_df, on='image_id')
sample_preds = merged.sample(n=min(6,len(merged)))
fig, axes = plt.subplots(2,3,figsize=(18,12))
axes = axes.ravel()
for idx,row in enumerate(sample_preds.iterrows()):
    img_path = f'{DATA_ROOT}/images/val/{row[1].image_id}'
    img = Image.open(img_path)
    axes[idx].imshow(img)
    title_lines = []
    for label in label_cols:
        gt = int(row[1][label])
        prob = row[1][f'prob_{label}']
        pred = 1 if prob>=0.5 else 0
        title_lines.append(f'{label}: GT={gt} Pred={pred} ({prob:.2f})')
    axes[idx].set_title('\n'.join(title_lines), fontsize=9, ha='left', loc='left')
    axes[idx].axis('off')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/project_stage2/sample_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Stage 3: Prepare Pointing Game CSV
import os, yaml, shutil, pandas as pd, cv2
ROBOFLOW_ROOT = '/content/drive/MyDrive/stage3_cam/'+dataset.location
OUTPUT_ROOT = '/content/drive/MyDrive/stage3_cam/data_stage3'
IMG_DST = os.path.join(OUTPUT_ROOT,'images')
ANN_DST = os.path.join(OUTPUT_ROOT,'annotations')
os.makedirs(IMG_DST,exist_ok=True)
os.makedirs(ANN_DST,exist_ok=True)
with open(os.path.join(ROBOFLOW_ROOT,'data.yaml'),'r') as f: yolo_data=yaml.safe_load(f)
class_names=yolo_data['names']
val_img_src=os.path.join(ROBOFLOW_ROOT,'valid','images')
val_label_src=os.path.join(ROBOFLOW_ROOT,'valid','labels')
rows=[]
for img_file in os.listdir(val_img_src):
    if not img_file.lower().endswith(('.jpg','.jpeg','.png')): continue
    shutil.copy2(os.path.join(val_img_src,img_file), os.path.join(IMG_DST,img_file))
    label_path=os.path.join(val_label_src,os.path.splitext(img_file)[0]+'.txt')
    if not os.path.exists(label_path): continue
    img=cv2.imread(os.path.join(val_img_src,img_file))
    if img is None: continue
    h,w=img.shape[:2]
    with open(label_path,'r') as f:
        for line in f:
            parts=line.strip().split()
            if len(parts)<5: continue
            class_id=int(parts[0])
            x=float(parts[1])*w
            y=float(parts[2])*h
            rows.append({'image_id':img_file,'class_id':class_id,'class_name':class_names[class_id],'x':x,'y':y})
points_df=pd.DataFrame(rows)
points_df.to_csv(os.path.join(ANN_DST,'pointing_game.csv'),index=False)
points_df.head()

In [ ]:
# Stage 3 Weak-CAM Inference
%cd /content/drive/MyDrive/stage3_cam/project_stage3
!PYTHONPATH=$PWD python -m src.stage3_localization.weak_cam.infer --config configs/stage3_weak_cam.yaml

In [ ]:
# Stage 3 Pointing Game Evaluation
!PYTHONPATH=$PWD python -m src.stage3_localization.weak_cam.eval_pointing_game --config configs/stage3_weak_cam_eval_stage1.yaml --data_root /content/drive/MyDrive/stage3_cam/data_stage3 --radius 15
!PYTHONPATH=$PWD python -m src.stage3_localization.weak_cam.eval_pointing_game --config configs/stage3_weak_cam_eval_stage2.yaml --data_root /content/drive/MyDrive/stage3_cam/data_stage3 --radius 15

In [ ]:
# Stage 3 Visualization of CAM + Points
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd, os
stage='stage1'
output_dir='/content/drive/MyDrive/stage3_cam/project_stage3/outputs/cam_results'
overlay_dir=os.path.join(output_dir,f'overlays_{stage}')
points_csv='/content/drive/MyDrive/stage3_cam/data_stage3/annotations/pointing_game.csv'
points_df=pd.read_csv(points_csv)
overlay_files=[f for f in os.listdir(overlay_dir) if f.endswith('.jpg')][:6]
fig, axes=plt.subplots(2,3,figsize=(18,12))
axes=axes.ravel()
for idx,img_file in enumerate(overlay_files):
    img_path=os.path.join(overlay_dir,img_file)
    img=Image.open(img_path)
    axes[idx].imshow(img)
    points=points_df[points_df['image_id']==img_file]
    for _,row in points.iterrows(): axes[idx].scatter(row['x'],row['y'],c='lime',s=50,marker='x',linewidths=2)
    axes[idx].set_title(f'{stage.upper()} CAM + Points: {img_file}',fontsize=10)
    axes[idx].axis('off')
plt.tight_layout()
plt.savefig(os.path.join(output_dir,f'cam_visualization_{stage}.png'),dpi=150,bbox_inches='tight')
plt.show()